In [1]:
#TASK 1-----GETTING THE LIST OF PLAYERS WHO HAVE EVER PLAYED AN ODI MATCH


#Importing required libraries

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv
import regex as re

#Nation ID'S 
nation_id={'Australia':2,'Bangladesh':25,'England':1,'India':6,
           'NewZealand':5,'Pakistan':7,'SouthAfrica':3,'SriLanka':8,
           'WestIndies':4,'Zimbabwe':9}

#Variables defined
df0=pd.DataFrame()
player_info=dict()

#Beautiful Soup
for nation,id in nation_id.items():    
    
    source = requests.get(f'http://www.espncricinfo.com/ci/content/player/caps.html?country={id};class=2').text
    soup = BeautifulSoup(source, 'lxml')

    for player_list in soup.find_all("li",{"class": "ciPlayername"}):
        
        player_name = player_list.a.text
        player_id = str(player_list.a)
        player_id = player_id.split('player/')[1]

        #player_id is unique for each player
        player_id = player_id.split('.html')[0]
        player_info.update({player_id:player_name})
        df = pd.DataFrame(data={'player_id':[player_id],'player_name':[player_name],'country':nation})
        df0 = df0.append(df)
       
   
#Exporting data to a csv file
df0.to_csv('player_list.csv',index=False)

#player_list contains all the names of ODI players
player_list = df0.reset_index(drop=True)

#player_info dictionary contains player_id & player_name as key-value pair
player_info = dict(zip(player_list.player_id[:],player_list.player_name))

In [109]:
#TASK 2--------LIST OF RUNS MADE BY PLAYERS EACH YEAR THEY HAVE PLAYED THE MATCH


id_list = list(player_info.keys())

#Variables defined
df2 = pd.DataFrame()
df21=pd.DataFrame()

#Beautiful Soup
for id in id_list:

    player_url = requests.get(f'http://stats.espncricinfo.com/ci/engine/player/{id}.html?class=2;template=results;type=batting').text
    sp = BeautifulSoup(player_url, 'lxml')

    th_count = 0
    #counting the block conatining runs detail in html code
    for count in sp.find_all("tr",{"class": "headlinks"}):
        th = str(count.find_all_next("th")).split("</th>")
        for thlis in th:
            th_count += 1
            if re.search("Runs",thlis):
                p = th_count
    #getting runs block
    for ls in sp.select("tr.data1>td.left"):
        if re.search("year",ls.text):
            year = str(ls.text).split("year")[1]
            year = year.strip()
            runs = ls.find_next_siblings("td")[p-2]
            runs_cleaned = str(runs.text).replace("-","0")
            
            #multi-indexing using id,year as tuple
            array = [[id],[year]]
            tuples = list(zip(*array))
            index = pd.MultiIndex.from_tuples(tuples,names = ['player_id','year'])
            df1 = pd.DataFrame(data = list(map(int,[runs_cleaned])),index = index,columns =['runs'])
            df2 = df2.append(df1,sort = True)
            


#Exporting data to a csv file grouped by id
df2.groupby(level=[0,1]).sum().reset_index().to_csv('runs_by_year_id.csv')

#Reading data from the csv file
runs_by_year_id = pd.read_csv('runs_by_year_id.csv')
runs_by_year_id = runs_by_year_id.set_index(['player_id','year']).drop(['Unnamed: 0'],axis=1)



runs_by_year_id.head()

runs
player_id year      
1043681   2018     7
10582     2007   304
          2008   297
          2009   436
          2010   103

In [110]:
#PLAYERS LIST AND THE TOTAL RUNS MADE BY THEM


total_runs = df2.groupby(level='player_id').apply(np.sum)
total_runs = total_runs.rename(index=str,columns={'runs': 'total_runs'})


players = pd.merge(total_runs,player_list,how='outer',on='player_id').sort_index(axis=1)
players=players.sort_values(by=['country']).reset_index(drop=True)
players.head()

,country,player_id,player_name,total_runs
0,Australia,7252,TD Paine,890
1,Australia,6995,SP O'Donnell,1242
2,Australia,7002,KJ O'Keeffe,16
3,Australia,7083,LS Pascoe,39
4,Australia,7115,WB Phillips,852


In [115]:
#TASK 3----------Cummulifying it all

grouped = df2.groupby(level=[0,1])

#funtion to get the total runs made by players

def total_runs_by_year(years,pl_id):    
    player_runs = 0
    for name,group in grouped:
        for plid in  pl_id:
            
            for year in years:
                
                if year == name[1] and plid == name[0]:
                    
                    player_runs+=grouped.get_group((plid,year))['runs'].sum()
       

    return(player_runs) 
pl_id= input("Enter player id/player id's seperated by space ")
pl_id = pl_id.split()

years = input("Enter year/years list seperated by space ")
years = years.split()

#calling function passing given inputs as parameter
total_runs_by_year(years,pl_id)

Enter player id/player id's seperated by space 35320
Enter year/years list seperated by space 2010 2011 2012


1032

In [111]:
#Unstacking levels

runs = runs_by_year_id.unstack(level=-1)
runs = runs.fillna(value='-')
runs.head()

runs                                               ...            \
year      1971 1972 1973 1974 1975 1976 1977 1978 1979 1980  ... 2010 2011   
player_id                                                    ...             
3943         -    -    -    -    -    -    -    -    -    -  ...    -    -   
3975         -    -    -    -    -    -    -    -    -    -  ...    -    -   
4112         -    -    -    -    -    -    -    -    -    -  ...    -    -   
4127         -    -    -    -    -    -    -    -    -    -  ...    -    -   
4144         -    -    -    -    -    -    -    -    -    -  ...    -    -   

                                                   
year      2012 2013 2014 2015 2016 2017 2018 2019  
player_id                                          
3943         -    -    -    -    -    -    -    -  
3975         -    -    -    -    -    -    -    -  
4112         -    -    -    -    -    -    -    -  
4127         -    -    -    -    -    -    -    -  
4144         -    -    -    -    -    -    -    -  

[5 rows x 49 columns]

In [112]:
#resetting index for merging operations
runs = runs.reset_index()
runs.head()

player_id runs                                          ...            \
year           1971 1972 1973 1974 1975 1976 1977 1978 1979  ... 2010 2011   
0         3943    -    -    -    -    -    -    -    -    -  ...    -    -   
1         3975    -    -    -    -    -    -    -    -    -  ...    -    -   
2         4112    -    -    -    -    -    -    -    -    -  ...    -    -   
3         4127    -    -    -    -    -    -    -    -    -  ...    -    -   
4         4144    -    -    -    -    -    -    -    -    -  ...    -    -   

                                              
year 2012 2013 2014 2015 2016 2017 2018 2019  
0       -    -    -    -    -    -    -    -  
1       -    -    -    -    -    -    -    -  
2       -    -    -    -    -    -    -    -  
3       -    -    -    -    -    -    -    -  
4       -    -    -    -    -    -    -    -  

[5 rows x 50 columns]

In [113]:
#converting player_id column values to str

runs['player_id'] = list(map(str,runs['player_id']))

In [4]:

#merging and obtaining final results

final = pd.merge(players,runs,how='outer',on='player_id')
final = final.sort_values(by='country')
final = final.reset_index(drop=True)

#Rearranging and drop columns to get the desired results
cols = list(final.columns)
a, b,  = cols.index('player_name'), cols.index('country')
cols[a], cols[b] = cols[b], cols[a]
final = final[cols]
final=final.drop(['player_id'],axis=1)
final.head()


,player_name,country,total_runs,"('runs', 1971)","('runs', 1972)","('runs', 1973)","('runs', 1974)","('runs', 1975)","('runs', 1976)","('runs', 1977)",...,"('runs', 2010)","('runs', 2011)","('runs', 2012)","('runs', 2013)","('runs', 2014)","('runs', 2015)","('runs', 2016)","('runs', 2017)","('runs', 2018)","('runs', 2019)"
0,TM Alderman,Australia,32,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,RJ Campbell,Australia,54,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,PH Carlson,Australia,11,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,GD Campbell,Australia,6,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4,IW Callen,Australia,6,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [6]:
final.to_csv('final.csv')